In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

#import os
#path_wd='drive/MyDrive/MINDAT Aspectos Avanzados/lulc-classification'
#os.chdir(path_wd)

In [5]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0']


Cargamos las bibliotecas necesarias

In [4]:
import numpy as np
import keras
import matplotlib.pyplot as plt
import keras.utils as np_utils
from keras.preprocessing.image import load_img,img_to_array
from keras.utils import np_utils
import cv2
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers  #Para definición de capas
from numpy.random import seed           #Para controlar generación de pesos
from tensorflow.keras.preprocessing.image import ImageDataGenerator #Para procesado de imágenes
import tensorflow.random

import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator #Para procesado de imágenes

from tensorflow.keras.applications.resnet import preprocess_input

Consideramos unas semillas aleatorias para garantizar reproducibilidad en los experimentos

In [ ]:
seed(1)
tensorflow.random.set_seed(0)

Fijamos los paths donde se encuentran las imagenes

In [ ]:
train_dir = "./Sentinel2LULC_354/Sentinel2LULC_354"
test_dir = "./Test 2/Test 2"
weights_file = None

Vamos a definir funciones auxiliares que nos ayudarán a visualizar diferentes aspectos

La primera función irá destinada a calcular la precisión total del modelo

In [ ]:
def calcularAccuracy(labels, preds):
  labels = np.argmax(labels, axis = 1)
  preds = np.argmax(preds, axis = 1)
  accuracy = sum(labels == preds)/len(labels)
  return accuracy

Vamos a hacer también una función para calcular la precisión por clase en el modelo

In [ ]:
def calcularAccuracyClases(labels, preds):
  well_pred = np.zeros(labels.shape[1]).tolist()
  total = np.zeros(labels.shape[1]).tolist()

  labels = np.argmax(labels, axis = 1)
  preds = np.argmax(preds, axis = 1)

  for i in len(labels):
    total[labels[i]] += 1
    if labels[i] == preds[i]:
      well_pred[labels[i]] += 1
  
  for i in len(well_pred):
    well_pred[i] /= float(total[i])
  
  return well_pred

Vamos a hacer una función también para visualizar la evolución de la medida de interés (accuracy en este caso) en entrenamiento y validación para cada época

In [ ]:
def mostrarEvolucion(hist):
  loss = hist.history['loss']
  val_loss = hist.history['val_loss']
  plt.plot(loss)
  plt.plot(val_loss)
  plt.legend(['Training loss', 'Validation loss'])
  plt.show()


  acc = hist.history['accuracy']
  val_acc = hist.history['val_accuracy']
  plt.plot(acc)
  plt.plot(val_acc)
  plt.legend(['Training accuracy','Validation accuracy'])
  plt.ylim(0,1)           #Meto esto para mejor comparación de gráficas
  plt.show()

Realizamos la lectura de las imagenes

In [ ]:
print("--- LECTURA DE IMÁGENES ---")

print("- Cargando imágenes de entrenamiento...")

x_train = []
y_train = []

for root, _, files in os.walk(train_dir):
    for file in files:
        if file.endswith(".jpg"):               
            x_train.append(cv2.imread(os.path.join(root, file), 1))
            y_train.append(int(file.split("_")[0]))

x_test = []
y_test = []
names_test = []

print("- Cargando imágenes de test...")

for root, _, files in os.walk(test_dir):
    for file in files:
        if file.endswith(".jpg"):               
            x_test.append(cv2.imread(os.path.join(root, file), 1))
            y_test.append(int(file.split("_")[0]))
            names_test.append(file)

# Convertir a numpy array
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
names_test = np.array(names_test)

# Normalizar las imágenes
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.0
x_test = x_test / 255.0

print(np.amax(x_train))

# Convertir a categóricas las etiquetas
y_train = y_train-1
y_test = y_test-1
y_train = np_utils.to_categorical(y_train, 29)
y_test = np_utils.to_categorical(y_test, 29)

Definimos ahora el modelo que vamos a utilizar

In [ ]:
 print("--- DEFINICIÓN DEL MODELO ---")
print("- Creando modelo...")
inputs = keras.Input(shape=(224,224,3))       #Las imágenes tendrán dimensión 224x224 y 3 bandas
# 224 * 224 * 3
conv1 = layers.Conv2D(6, 5, activation="relu")(inputs)      #Al hacerlo así, estamos aplicando la convolución y luego la activación
# 220 * 220 * 6
maxpooling1 = layers.MaxPooling2D()(conv1)
# 110 * 110 * 6
conv2 = layers.Conv2D(16, 3, activation="relu")(maxpooling1)
# 108 * 108 * 16
maxpooling2 = layers.MaxPooling2D()(conv2)
# 54 * 54 * 16
conv3 = layers.Conv2D(32, 3, activation="relu")(maxpooling2)
# 52 * 52 * 32
maxpooling3 = layers.MaxPooling2D()(conv3)
# 26 * 26 * 32
conv4 = layers.Conv2D(40, 3, activation="relu")(maxpooling3)
# 24 * 24 * 40
maxpooling4 = layers.MaxPooling2D()(conv4)
# 12 * 12 * 40
conv5 = layers.Conv2D(50, 3, activation="relu")(maxpooling4)
# 10 * 10 * 50
maxpooling5 = layers.MaxPooling2D()(conv5)
# 5 * 5 * 50

flatten = layers.Flatten()(maxpooling5)                 #Convierte la información a array de 5 * 5 * 50 = 1250 posiciones
# 1250
dense1 = layers.Dense(50, activation="relu")(flatten)
# 50
outputs = layers.Dense(29, activation="softmax")(dense1)
# 29 -> probabilidades de pertenecer a cada clase

#Ahora asociamos la estructura a nuestro modelo
model = keras.Model(inputs=inputs, outputs=outputs)

#Imprimimos ahora información del modelo para ver que es correcta.
model.summary()

#Ponemos inicialmente el optimizador que nos viene dado en las diapos de prácticas
opt = tf.keras.optimizers.SGD(lr=0.01, decay= 1e-6, momentum=0.9, nesterov=True)

#La función de pérdida que utilizaremos será la categorical_crossentropy.
#Compilamos el modelo.
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])


# Una vez tenemos el modelo base, si se indica un archivo de pesos para inicializar,
# los vamos a cargar
if not weights_file is None:
    model.load_weights(weights_file)

Ejecutamos el modelo

In [ ]:
print("--- ENTRENAMIENTO DEL MODELO ---")
p_train = np.random.permutation(len(x_train))
p_test = np.random.permutation(len(x_test))

x_train = x_train[p_train]
y_train = y_train[p_train]
x_test = x_test[p_test]
y_test = y_test[p_test]
names_test = names_test[p_test]

print("- Entrenando el modelo...")
history = model.fit(x_train, y_train, batch_size= 32, epochs=10, validation_split=0.1)
model.save_weights('my_model_weights.h5')
mostrarEvolucion(history)

Realizamos la predicción sobre el conjunto de test

In [ ]:
pred = model.predict(x_test,verbose=1)
print(pred)
    


Calculamos algunas medidas de interés sobre la predicción realizada

In [ ]:
print("Accuracy on test: " + str(calcularAccuracy(y_test, pred)))
pred = np.argmax(pred, axis = 1)
print(pred)
pred = pred+1

results = pd.DataFrame({'id.jpg': names_test, 'label': pred})
results.to_csv('pred1.csv', index=False)